<a href="https://colab.research.google.com/github/WZRP/mass-watermarker/blob/main/Mass_Watermarker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Please open Colab Files Explorer to Upload the Watermark Image yourself.

In [ ]:
#@title # Download ZIP of images, then UNZIP to folder
import os
import io
import requests
import zipfile

# Define the URL of the file to download
url = "" #@param {'type': 'string'}

# Define the name of the local directory to extract to
extract_dir = "" #@param {'type': 'string'}

# Create the directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Download the file and save it to memory
response = requests.get(url)
data = response.content

# Extract the contents of the ZIP file to the directory
with zipfile.ZipFile(io.BytesIO(data)) as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
#@title # Convert all Images to PNG
import os
from PIL import Image

# Define the directory path
dir_path = "" #@param {'type': 'string'}

# Loop through all files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is an image (JPG or JPEG)
    if filename.lower().endswith(('.jpg', '.jpeg')):
        # Rename the file to have a .png extension
        os.rename(os.path.join(dir_path, filename), os.path.join(dir_path, os.path.splitext(filename)[0] + ".png"))

In [ ]:
#@title # Mass Rename Images to 0000 -> 9999
import os

# Define the directory path
folder_dir = ""  #@param {'type': 'string'}

# Define the starting number for the renaming
start_num = 0

# Loop through all files in the directory
for filename in os.listdir(folder_dir):
    # Check if the file is an image (JPG, JPEG, or PNG)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Generate the new filename with a four-digit number
        new_filename = f"{start_num:04}.png"
        # Rename the file with the new filename
        os.rename(os.path.join(folder_dir, filename), os.path.join(folder_dir, new_filename))
        # Increment the start number for the next file
        start_num += 1

In [ ]:
#@title # Image Dimension Viewer
import os
from PIL import Image

# Define the input directory path
input_dir = "/content/raw"

# Create an empty list to store the dimensions of all images
dimensions = []

# Loop through all files in the input directory
for filename in os.listdir(input_dir):
    # Check if the file is an image (JPEG or PNG)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Open the image file
        input_filepath = os.path.join(input_dir, filename)
        with Image.open(input_filepath) as img:
            # Get the dimensions of the image
            width, height = img.size
            # Add the filename and dimensions to the list
            dimensions.append((filename, width, height))

# Print the list of filenames and dimensions
for filename, width, height in dimensions:
    print(f"{filename}: {width}x{height}")

In [ ]:
#@title # Mass Insert Watermark (Upgraded)
import os
import tqdm
from PIL import Image

# Watermark image file
watermark_file = '' #@param {'type': 'string'}

# Folder of images to watermark
images_folder = '' #@param {'type': 'string'}

# Output folder for watermarked images
output_folder = '' #@param {'type': 'string'}


# Create output folder if doesn't exist
if not os.path.exists(output_folder):
  os.makedirs(output_folder)

# Load watermark image
watermark = Image.open(watermark_file)

# Get number of images to process
num_images = len(os.listdir(images_folder))

# Sort image names alphabetically
image_files = sorted(os.listdir(images_folder))

# Process images with progress bar
with tqdm.tqdm(total=num_images) as pbar:
  for i, image_file in enumerate(image_files):

    # Open image
    try:
      image = Image.open(os.path.join(images_folder, image_file))
    except OSError as e:
      print(f"Error opening {image_file}: {e}")
      continue

    # Watermark image
    w, h = watermark.size
    padding = 128
    image.paste(watermark, (padding, image.height - h - padding), watermark)

    # Save watermarked image as PNG
    output_path = os.path.join(output_folder, os.path.splitext(image_file)[0] + '.png')
    image.save(output_path)

    # Update progress bar
    pbar.update(1)

print('\nWatermarked images saved to', output_folder)

In [ ]:
#@title # ZIP Folder
import os
import zipfile
import tqdm

# Define the input directory path, the desired name of the ZIP file,
# and the desired output directory for the ZIP file
input_dir = "" #@param {'type': 'string'}
zip_filename = "" #@param {'type': 'string'}
output_dir = "" #@param {'type': 'string'}

# Create a new ZIP file
output_zip = os.path.join(output_dir, zip_filename)
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zip_file:
    # Get the number of files to add to the ZIP archive
    num_files = sum(len(files) for _, _, files in os.walk(input_dir))

    # Loop through all files in the input directory and add them to the ZIP archive
    with tqdm.tqdm(total=num_files) as pbar:
        for root, dirs, files in os.walk(input_dir):
            for file in files:
                # Get the full path of the file
                file_path = os.path.join(root, file)
                # Add the file to the ZIP archive
                zip_file.write(file_path, arcname=os.path.relpath(file_path, input_dir))
                # Update the progress bar
                pbar.update(1)

print(f"\nZIP archive created successfully at {output_zip}")